In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('./Bilan tarification 2019.csv', sep=";", encoding="iso-8859-1")
df

In [ ]:
df[df['Id Famille'] ==  443]

In [ ]:
df[['Id Personne', 'Id Famille', 'QF', 'Type repas', 'Nombre']]

In [ ]:
df[['Type repas', 'Nombre']].groupby('Type repas').count()

In [ ]:
df[['Type repas', 'Nombre']].groupby('Type repas').sum()

In [ ]:
np.unique(df['Id Famille'])

In [ ]:
df['Repas'] = np.select(
    [
        df["Type repas"].str.contains('Végétarien'),
        df["Type repas"].str.contains('Panier'),
    ],
    [
        'cantine_vegetarien',
        'cantine_panier',
    ],
    default = 'cantine'
)
df

In [ ]:
df[['Repas', 'Nombre']]

In [ ]:
df[['Repas', 'Nombre']].groupby('Repas').count()

In [ ]:
df[['Repas', 'Nombre']].groupby('Repas').sum()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(df[['Repas']])

In [ ]:
plt.hist(df[['Repas']], weights = df[['Nombre']])

In [ ]:
df.QF

In [ ]:
np.unique(df.QF)

In [ ]:
plt.hist(df.QF)

In [ ]:
plt.hist(df.QF[df.QF< 6000])

In [ ]:
plt.hist(df.QF[df.QF< 6000], density=True, bins=100, cumulative=True)

In [ ]:
plt.hist(df.QF[df.QF< 6000], density=True, bins=100, cumulative=True, weights=df.Nombre[df.QF< 6000])

In [ ]:
plt.hist(df.QF[df.QF< 6000], density=True, bins=100, weights=df.Nombre[df.QF< 6000])

In [ ]:
plt.hist(df.QF[df.QF< 6000], cumulative=True, weights=df.Nombre[df.QF< 6000])

In [ ]:
plt.hist(df.QF[df.QF< 6000], bins=[0, 410, 520, 6000], weights=df.Nombre[df.QF< 6000])

In [ ]:
inf, sup = (0, 410)
cond = (inf < df.QF) & (df.QF < sup)
plt.hist(df.QF[cond], bins=100, weights=df.Nombre[cond])

In [ ]:
inf, sup = (410, 520)
cond = (inf < df.QF) & (df.QF < sup)
plt.hist(df.QF[cond], bins=10, weights=df.Nombre[cond])

In [ ]:
inf, sup = (410, 520)
cond = (inf < df.QF) & (df.QF < sup)
plt.hist(df.QF[cond], bins=10, weights=df.Nombre[cond])

In [ ]:
df[df['Id Famille'] == 443]

In [ ]:
df.QF = np.minimum(df.QF, 99999)

In [ ]:
bogus_raw_df = pd.pivot_table(df, index=["Id Personne", "Id Famille", "QF"], columns="Repas", values="Nombre", fill_value=0)
bogus_raw_df

In [ ]:
raw_df = pd.pivot_table(df, index=["Id Personne", "Id Famille", "QF"], columns="Repas", values="Nombre", fill_value=0, aggfunc=np.sum)
#raw_df[fix_raw_df.index == (6691, 443, 1030)]

In [ ]:
raw_df[raw_df.index == (6691, 443, 1030)]

In [ ]:
flat_df = raw_df.reset_index()
flat_df

In [ ]:
flat_df[flat_df['Id Famille'] == 443]

In [ ]:
familles_ids = flat_df["Id Famille"] * 100000 + flat_df.QF
familles_ids

In [ ]:
(unique_famille_ids, famille_idx, famille_id_inv) = np.unique(familles_ids, return_index=True, return_inverse=True)
unique_famille_ids, famille_idx, famille_id_inv

In [ ]:
unique_famille_ids

In [ ]:
len(famille_id_inv)

In [ ]:
len(famille_idx)

In [ ]:
famille_idx[famille_id_inv]

In [ ]:
unique_famille_ids

In [ ]:
len(unique_famille_ids)

In [ ]:
famille_id_inv

In [ ]:
individu_df = pd.DataFrame({
    'famille_id': famille_id_inv,
    'strasbourg_metropole_nombre_repas_cantine':flat_df.cantine,
    'strasbourg_metropole_nombre_repas_cantine_vegetarien':flat_df.cantine_vegetarien,
    'strasbourg_metropole_nombre_repas_cantine_panier':flat_df.cantine_panier,
})
individu_df

In [ ]:
famille_idx

In [ ]:
famille_df = pd.DataFrame({
    'strasbourg_metropole_quotient_familial': flat_df.QF[famille_idx]
})
famille_df

In [ ]:
base_year = "2021-03"

In [ ]:
from openfisca_survey_manager.scenarios import AbstractSurveyScenario
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
from openfisca_core import periods
from openfisca_core.model_api import Reform

In [ ]:
def modify_parameters(parameters):
    pp = parameters.metropoles.strasbourg.tarifs_cantine
    pp[1].amount.update(start = periods.period("2021-01-01"), value = 1.5)
    pp[2].amount.update(start = periods.period("2021-01-01"), value = 1.5)
    return parameters
    
class reform_fusion_1_3(Reform):
    name = u"Fusionne les trois premières tranches"

    def apply(self):
        self.modify_parameters(modifier_function = modify_parameters)

In [ ]:
class StrasbourgSurveyScenario(AbstractSurveyScenario):
    def __init__(self, data = None, reform = None):
        super(StrasbourgSurveyScenario, self).__init__()

        base = CountryTaxBenefitSystem()
        base.load_extension('openfisca_france_local')
        if reform is not None:
            tax_benefit_system = reform(base)
        else:
            tax_benefit_system = base
        self.year = base_year

        if 'input_data_frame_by_entity_by_period' in data:
            period = periods.period(self.year)
            dataframe_variables = set()
            for entity_dataframe in data['input_data_frame_by_entity_by_period'][period].values():
                if not isinstance(entity_dataframe, pd.DataFrame):
                    continue
                dataframe_variables = dataframe_variables.union(set(entity_dataframe.columns))
            self.used_as_input_variables = list(
                set(tax_benefit_system.variables.keys()).intersection(dataframe_variables)
                )

        self.set_tax_benefit_systems(tax_benefit_system)
        self.init_from_data(data = data)

In [ ]:
individu_df['famille_role_index'] = 0
individu_df['foyer_fiscal_id'] = individu_df.famille_id
individu_df['foyer_fiscal_role_index'] = 0
individu_df['menage_id'] = individu_df.famille_id
individu_df['menage_role_index'] = 0

In [ ]:
menage_df = pd.DataFrame({})
foyerfiscaux_df = pd.DataFrame({})

In [ ]:
periods.period(base_year)

In [ ]:
data = dict(input_data_frame_by_entity_by_period = {periods.period(base_year): dict(
individu=individu_df,
famille=famille_df,
menage=menage_df,
foyer_fiscal=foyerfiscaux_df)})

In [ ]:
def compute_result(data, reform):
    scenario = StrasbourgSurveyScenario(data = data, reform = reform)
    cout_cantine = scenario.simulation.calculate('strasbourg_metropole_cout_cantine', period = '2021-03')
    cout_cantine_vege = scenario.simulation.calculate('strasbourg_metropole_cout_cantine_repas_vegetarien', period = '2021-03')
    cout_cantine_panier = scenario.simulation.calculate('strasbourg_metropole_cout_cantine_repas_panier', period = '2021-03')
    return (scenario, (sum(cout_cantine), sum(cout_cantine_vege), sum(cout_cantine_panier)))

In [ ]:
base = compute_result(data, None)

In [ ]:
fusion = compute_result(data, reform_fusion_1_3)

In [ ]:
base[0]

In [ ]:
base[1]

In [ ]:
base[0].tax_benefit_system.parameters.metropoles.strasbourg.tarifs_cantine

In [ ]:
scenario = StrasbourgSurveyScenario(data = data)

In [ ]:
tarif_cantine = scenario.simulation.calculate('strasbourg_metropole_tarification_cantine', period = '2021-03')
tarif_cantine

In [ ]:
cout_cantine = scenario.simulation.calculate('strasbourg_metropole_cout_cantine', period = '2021-03')
cout_cantine

In [ ]:
cout_cantine = scenario.simulation.calculate('strasbourg_metropole_cout_cantine', period = '2021-03')
cout_cantine

In [ ]:
sum(cout_cantine)

In [ ]:
sum(cout_cantine)

In [ ]:
len(cout_cantine)


In [ ]:
famille_df['uid'] = unique_famille_ids
famille_df['cout_cantine'] = cout_cantine
famille_df

In [ ]:
famille_df[famille_df.uid == 44301030]

In [ ]:
individu_df

In [ ]:
flat_df

In [ ]:
flat_df[flat_df['Id Famille'] == 443]

In [ ]:
famille_df.to_csv('openfisca.csv')

In [ ]:
cout_cantine_vege = scenario.simulation.calculate('strasbourg_metropole_cout_cantine_repas_vegetarien', period = '2021-03')
cout_cantine_vege

In [ ]:
sum(cout_cantine_vege)

In [ ]:
cout_cantine_panier = scenario.simulation.calculate('strasbourg_metropole_cout_cantine_repas_panier', period = '2021-03')
cout_cantine_panier

In [ ]:
sum(cout_cantine_panier)

In [ ]:
len(cout_cantine)

In [ ]:
flat_df

In [ ]:
e = [1,2,3]

In [ ]:
e.index(3)

In [ ]:
p = scenario.tax_benefit_system.get_parameters_at_instant('2021-04').metropoles.strasbourg.tarifs_cantine
p

In [ ]:
p.thresholds

In [ ]:
p.amounts

In [ ]:
xmax = 2500

In [ ]:
plt.step(p.thresholds + [xmax], p.amounts + [p.amounts[-1]], where='post')
plt.xlim(xmin=0)

In [ ]:
df.CapQF = np.minimum(df.QF, xmax)
res = plt.hist(df.CapQF, bins=p.thresholds + [xmax], weights=df.Nombre)

In [ ]:
plt.hist(df.CapQF, bins=100, weights=df.Nombre)

In [ ]:
res = plt.hist(df.CapQF, bins=p.thresholds + [xmax], density=True, weights=df.Nombre)

In [ ]:
plt.hist(df.CapQF, bins=p.thresholds + [xmax], density=True, cumulative=True, weights=df.Nombre)

In [ ]:
res = plt.hist(df.CapQF, bins=p.thresholds + [xmax], weights=df.Nombre, cumulative=True)